In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from tqdm import tqdm
from sklearn.metrics import f1_score

from Bio.SeqUtils.ProtParam import ProteinAnalysis

import optuna
from xgboost import XGBClassifier
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from tqdm import tqdm

# 전체 코드 process

    1. 데이터 전 처리
        - disease_dic 생성, train 과정에 없는 disease들은 0으로 unknown 토큰 처리
        - 결측치 train 데이터 기반으로 채우기 및 정규화
        
    2. Input_feautres
        - Antigen eatures
        - Peptide Epitope features
        - Disease features
        
    3. XGBoost 
        - Optuna 기반 최적의 하이퍼 파라미터 탐색
        - Feature importance 계산

# Data preprocessing

In [5]:
#train 데이터 불러오기
train = pd.read_csv('../data/train.csv')

# Disease Features

In [6]:
# disease dic 생성
def get_dic(data):
    vocab = {}
    for name in data:
        if name not in vocab:
            vocab[name]=0
        vocab[name] += 1
    vocab_sorted = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
    token_dic = {}
    i = 1
    # train에서 보지 않은 disease들 Unknown 토큰 처리
    # train과정에서 없는 disease들은 0으로 처리하였음
    token_dic['Unknown'] = 0
    for (name, freq) in vocab_sorted:
        token_dic[name] = i
        i += 1
    return token_dic

def dic_except(dic, a):
    try:
        return dic[a]
    except:
        return dic['Unknown']

dic_disease_type = get_dic(train['disease_type'])
dic_disease_state = get_dic(train['disease_state'])

train['disease_type'] = train['disease_type'].map(lambda a: dic_except(dic_disease_type, a))
train['disease_state'] = train['disease_state'].map(lambda a: dic_except(dic_disease_state, a))

# Epitope Peptide Features

In [7]:
# 총 7가지 peptide features를 계산

def get_peptide_feature(seq): # CTD descriptor
    CTD = {'hydrophobicity': {1: ['R', 'K', 'E', 'D', 'Q', 'N'], 2: ['G', 'A', 'S', 'T', 'P', 'H', 'Y'], 3: ['C', 'L', 'V', 'I', 'M', 'F', 'W']},
           'normalized.van.der.waals': {1: ['G', 'A', 'S', 'T', 'P', 'D', 'C'], 2: ['N', 'V', 'E', 'Q', 'I', 'L'], 3: ['M', 'H', 'K', 'F', 'R', 'Y', 'W']},
           'polarity': {1: ['L', 'I', 'F', 'W', 'C', 'M', 'V', 'Y'], 2: ['P', 'A', 'T', 'G', 'S'], 3: ['H', 'Q', 'R', 'K', 'N', 'E', 'D']},
           'polarizability': {1: ['G', 'A', 'S', 'D', 'T'], 2: ['C', 'P', 'N', 'V', 'E', 'Q', 'I', 'L'], 3: ['K', 'M', 'H', 'F', 'R', 'Y', 'W']},
           'charge': {1: ['K', 'R'], 2: ['A', 'N', 'C', 'Q', 'G', 'H', 'I', 'L', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V'], 3: ['D', 'E']},
           'secondary': {1: ['E', 'A', 'L', 'M', 'Q', 'K', 'R', 'H'], 2: ['V', 'I', 'Y', 'C', 'W', 'F', 'T'], 3: ['G', 'N', 'P', 'S', 'D']},
           'solvent': {1: ['A', 'L', 'F', 'C', 'G', 'I', 'V', 'W'], 2: ['R', 'K', 'Q', 'E', 'N', 'D'], 3: ['M', 'S', 'P', 'T', 'H', 'Y']}}
    
    seq = str(seq)
    sequencelength = len(seq)
    Sequence_group = []
    
    for AAproperty in CTD:
        propvalues = ""
        for letter in seq:
            if letter in CTD[AAproperty][1]:
                propvalues += "1"
            elif letter in CTD[AAproperty][2]:
                propvalues += "2"
            elif letter in CTD[AAproperty][3]:
                propvalues += "3"
        abpos_1 = [i for i in range(len(propvalues)) if propvalues.startswith("1", i)]
        abpos_1 = [x+1 for x in abpos_1]
        abpos_1.insert(0, "-")
        abpos_2 = [i for i in range(len(propvalues)) if propvalues.startswith("2", i)]
        abpos_2 = [x+1 for x in abpos_2]
        abpos_2.insert(0, "-")
        abpos_3 = [i for i in range(len(propvalues)) if propvalues.startswith("3", i)]
        abpos_3 = [x+1 for x in abpos_3]
        abpos_3.insert(0, "-")
        property_group1_length = propvalues.count("1")
        
        if property_group1_length == 0:
            Sequence_group.extend([0, 0, 0, 0, 0])
        elif property_group1_length == 1:
            Sequence_group.append((abpos_1[1]/sequencelength)*100)
            Sequence_group.append((abpos_1[1]/sequencelength)*100)
            Sequence_group.append((abpos_1[1]/sequencelength)*100)
            Sequence_group.append((abpos_1[1]/sequencelength)*100)
            Sequence_group.append((abpos_1[1]/sequencelength)*100)
        elif property_group1_length == 2:
            Sequence_group.append((abpos_1[1]/sequencelength)*100)
            Sequence_group.append((abpos_1[1]/sequencelength)*100)
            Sequence_group.append((abpos_1[round((0.5*property_group1_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_1[round((0.75*property_group1_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_1[property_group1_length]/sequencelength)*100)
        else:
            Sequence_group.append((abpos_1[1]/sequencelength)*100)
            Sequence_group.append((abpos_1[round((0.25*property_group1_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_1[round((0.5*property_group1_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_1[round((0.75*property_group1_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_1[property_group1_length]/sequencelength)*100)

        property_group2_length = propvalues.count("2")
        if property_group2_length == 0:
            Sequence_group.extend([0, 0, 0, 0, 0])
        elif property_group2_length == 1:
            Sequence_group.append((abpos_2[1]/sequencelength)*100)
            Sequence_group.append((abpos_2[1]/sequencelength)*100)
            Sequence_group.append((abpos_2[1]/sequencelength)*100)
            Sequence_group.append((abpos_2[1]/sequencelength)*100)
            Sequence_group.append((abpos_2[1]/sequencelength)*100)
        elif property_group2_length == 2:
            Sequence_group.append((abpos_2[1]/sequencelength)*100)
            Sequence_group.append((abpos_2[1]/sequencelength)*100)
            Sequence_group.append((abpos_2[round((0.5*property_group2_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_2[round((0.75*property_group2_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_2[property_group2_length]/sequencelength)*100)
        else:
            Sequence_group.append((abpos_2[1]/sequencelength)*100)
            Sequence_group.append((abpos_2[round((0.25*property_group2_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_2[round((0.5*property_group2_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_2[round((0.75*property_group2_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_2[property_group2_length]/sequencelength)*100)

        property_group3_length = propvalues.count("3")
        if property_group3_length == 0:
            Sequence_group.extend([0, 0, 0, 0, 0])
        elif property_group3_length == 1:
            Sequence_group.append((abpos_3[1]/sequencelength)*100)
            Sequence_group.append((abpos_3[1]/sequencelength)*100)
            Sequence_group.append((abpos_3[1]/sequencelength)*100)
            Sequence_group.append((abpos_3[1]/sequencelength)*100)
            Sequence_group.append((abpos_3[1]/sequencelength)*100)
        elif property_group3_length == 2:
            Sequence_group.append((abpos_3[1]/sequencelength)*100)
            Sequence_group.append((abpos_3[1]/sequencelength)*100)
            Sequence_group.append((abpos_3[round((0.5*property_group3_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_3[round((0.75*property_group3_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_3[property_group3_length]/sequencelength)*100)
        else:
            Sequence_group.append((abpos_3[1]/sequencelength)*100)
            Sequence_group.append((abpos_3[round((0.25*property_group3_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_3[round((0.5*property_group3_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_3[round((0.75*property_group3_length)-0.1)]/sequencelength)*100)
            Sequence_group.append((abpos_3[property_group3_length]/sequencelength)*100)
    return Sequence_group

# Antigen Features

In [ ]:
# Bio python 툴을 이용하여 Antigen의 4가지 features extraction

def get_protein_feature(seq):
    protein_feature = []
    protein_feature.append(ProteinAnalysis(seq).isoelectric_point())
    protein_feature.append(ProteinAnalysis(seq).aromaticity())
    protein_feature.append(ProteinAnalysis(seq).gravy())
    protein_feature.append(ProteinAnalysis(seq).instability_index())
    return protein_feature

# Get Features

In [8]:

def get_preprocessing(data_type, new_df):   
    protein_features = []
    epitope_features = []
    disease_features = []
        
    for epitope, antigen, d_type, d_state in tqdm(zip(new_df['epitope_seq'], new_df['antigen_seq'], new_df['disease_type'], new_df['disease_state'])):        

        protein_features.append(get_protein_feature(antigen))
        epitope_features.append(get_peptide_feature(epitope))
        disease_features.append([d_type, d_state])
    
    label_list = None
    if data_type != 'test':
        label_list = []
        for label in new_df['label']:
            label_list.append(label)
    print(f'{data_type} dataframe preprocessing was done.')
    return protein_features, epitope_features, disease_features, label_list

# Train & Validation Split

In [9]:
train, val = train_test_split(train, train_size=0.8, random_state=12)

train_protein_features, train_epitope_features, train_disease_features, train_label_list = get_preprocessing('train', train)
val_protein_features, val_epitope_features, val_disease_features, val_label_list = get_preprocessing('val', val)

152648it [05:29, 463.09it/s]
93it [00:00, 457.20it/s]

train dataframe preprocessing was done.


38163it [01:21, 466.68it/s]

val dataframe preprocessing was done.


In [10]:
train_protein_features = np.array(train_protein_features)
train_epitope_features = np.array(train_epitope_features)
train_disease_features = np.array(train_disease_features)
X_train = np.concatenate((train_protein_features, train_epitope_features, train_disease_features), axis=1)
y_train = np.array(train_label_list)

val_protein_features = np.array(val_protein_features)
val_epitope_features = np.array(val_epitope_features)
val_disease_features = np.array(val_disease_features)
X_val = np.concatenate((val_protein_features, val_epitope_features, val_disease_features), axis=1)
y_val = np.array(val_label_list)

# XGBoost Optimization

In [14]:
# XGBoost 머신 러닝 모델 하이퍼 파라미터 최적화

def objective(trial: Trial) -> float:
    param = {'verbosity':1,
             'objective':'binary:logistic', #
             'max_depth':trial.suggest_int('max_depth',3,30),
             'learning_rate':trial.suggest_loguniform('learning_rate',1e-8,1e-2),
             'n_estimators':trial.suggest_int('n_estimators',100,3000),
             'subsample':trial.suggest_loguniform('subsample',0.7,1),
             'min_child_weight': trial.suggest_int('min_child_weight', 1, 300 ),
             'alpha': trial.suggest_loguniform( 'alpha', 1e-3, 10.0),
             'random_state': 42}
    model = XGBClassifier(**param)
    model.fit(
        np.array(X_train),
        np.array(y_train),
        eval_set=[(np.array(X_train), np.array(y_train)), (np.array(X_val), np.array(y_val))],
        early_stopping_rounds=100,
        eval_metric = 'logloss',
        verbose=False
    )

    pred = model.predict(np.array(X_val))
    log_score = log_loss(np.array(y_val), pred)
    
    return log_score

sampler = TPESampler(seed=42)
studyXGB = optuna.create_study(

    study_name="XGboost",
    direction="minimize",
    sampler=sampler,
)

studyXGB.optimize(objective, n_trials=100)
print("Best Score:", studyXGB.best_value)
print("Best trial:", studyXGB.best_trial.params)

[I 2022-07-28 14:15:55,272] A new study created in memory with name: XGboost
[I 2022-07-28 14:21:36,712] Trial 0 finished with value: 1.996512824902588 and parameters: {'max_depth': 13, 'learning_rate': 0.005061576888752304, 'n_estimators': 2223, 'subsample': 0.8666253978417815, 'min_child_weight': 47, 'alpha': 0.004207053950287938}. Best is trial 0 with value: 1.996512824902588.
[I 2022-07-28 14:23:27,901] Trial 1 finished with value: 2.5359083968092255 and parameters: {'max_depth': 4, 'learning_rate': 0.0015741890047456662, 'n_estimators': 1843, 'subsample': 0.9011142760532918, 'min_child_weight': 7, 'alpha': 7.579479953348009}. Best is trial 0 with value: 1.996512824902588.
[I 2022-07-28 14:25:06,387] Trial 2 finished with value: 2.636376431289911 and parameters: {'max_depth': 26, 'learning_rate': 1.879466824163843e-07, 'n_estimators': 627, 'subsample': 0.7473219839293679, 'min_child_weight': 92, 'alpha': 0.12561043700013558}. Best is trial 0 with value: 1.996512824902588.
[I 2022-0

[I 2022-07-28 16:25:37,270] Trial 26 finished with value: 2.6264192867749205 and parameters: {'max_depth': 6, 'learning_rate': 0.00014613518661471525, 'n_estimators': 1952, 'subsample': 0.8076928138332339, 'min_child_weight': 107, 'alpha': 0.24444361636070627}. Best is trial 12 with value: 1.9404011946707964.
[I 2022-07-28 16:32:04,358] Trial 27 finished with value: 2.16846818304388 and parameters: {'max_depth': 13, 'learning_rate': 0.0018533339988984203, 'n_estimators': 2342, 'subsample': 0.7677871725359959, 'min_child_weight': 62, 'alpha': 0.011624806031171957}. Best is trial 12 with value: 1.9404011946707964.
[I 2022-07-28 16:40:14,688] Trial 28 finished with value: 2.3431413222206188 and parameters: {'max_depth': 28, 'learning_rate': 5.111031250055173e-05, 'n_estimators': 2235, 'subsample': 0.8821351587386572, 'min_child_weight': 48, 'alpha': 0.0022355661686797495}. Best is trial 12 with value: 1.9404011946707964.
[I 2022-07-28 16:46:41,219] Trial 29 finished with value: 2.04900285

[I 2022-07-28 18:42:53,081] Trial 52 finished with value: 1.9078184543090166 and parameters: {'max_depth': 15, 'learning_rate': 1.0802110584195243e-07, 'n_estimators': 1525, 'subsample': 0.702696954661618, 'min_child_weight': 1, 'alpha': 0.018680769145290464}. Best is trial 36 with value: 1.8489919553925567.
[I 2022-07-28 18:43:17,723] Trial 53 finished with value: 2.2834094550167126 and parameters: {'max_depth': 14, 'learning_rate': 9.114270043951241e-08, 'n_estimators': 105, 'subsample': 0.7048132433933325, 'min_child_weight': 20, 'alpha': 0.019501157111050386}. Best is trial 36 with value: 1.8489919553925567.
[I 2022-07-28 18:46:27,754] Trial 54 finished with value: 2.305130164475826 and parameters: {'max_depth': 18, 'learning_rate': 3.823443098333927e-08, 'n_estimators': 1614, 'subsample': 0.7405374920856426, 'min_child_weight': 34, 'alpha': 0.03512511191478571}. Best is trial 36 with value: 1.8489919553925567.
[I 2022-07-28 18:48:38,318] Trial 55 finished with value: 2.14403306584

[I 2022-07-28 20:50:44,927] Trial 78 finished with value: 2.241777997443315 and parameters: {'max_depth': 22, 'learning_rate': 4.207419466154609e-06, 'n_estimators': 1902, 'subsample': 0.7659039403031009, 'min_child_weight': 20, 'alpha': 0.0018900052313522364}. Best is trial 71 with value: 1.8182210411181898.
[I 2022-07-28 20:56:59,666] Trial 79 finished with value: 2.5413446293493656 and parameters: {'max_depth': 21, 'learning_rate': 0.0004968165668724602, 'n_estimators': 2075, 'subsample': 0.7095075388467996, 'min_child_weight': 151, 'alpha': 0.002451649787230088}. Best is trial 71 with value: 1.8182210411181898.
[I 2022-07-28 21:01:28,333] Trial 80 finished with value: 2.036334363035486 and parameters: {'max_depth': 26, 'learning_rate': 0.0003543920422135956, 'n_estimators': 865, 'subsample': 0.7543652958697858, 'min_child_weight': 9, 'alpha': 0.11411889509395365}. Best is trial 71 with value: 1.8182210411181898.
[I 2022-07-28 21:02:15,460] Trial 81 finished with value: 3.1368445610

Best Score: 1.7874488487618263
Best trial: {'max_depth': 16, 'learning_rate': 0.0025640961747953163, 'n_estimators': 1546, 'subsample': 0.7352711832885261, 'min_child_weight': 1, 'alpha': 0.027937140843060745}


### Best trial: {'max_depth': 16, 'learning_rate': 0.0025640961747953163, 'n_estimators': 1546, 'subsample': 0.7352711832885261, 'min_child_weight': 1, 'alpha': 0.027937140843060745}

# Calculating Features Importance 

In [17]:
importances = model.feature_importances_

indices = np.argsort(importances)[::-1]

print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("{}. feature {} ({:.3f})".format(f + 1, indices[f], importances[indices[f]]))

Feature ranking:
1. feature 110 (0.241)
2. feature 109 (0.047)
3. feature 69 (0.033)
4. feature 1 (0.015)
5. feature 0 (0.014)
6. feature 2 (0.014)
7. feature 78 (0.014)
8. feature 38 (0.012)
9. feature 3 (0.012)
10. feature 79 (0.010)
11. feature 68 (0.009)
12. feature 4 (0.009)
13. feature 34 (0.009)
14. feature 75 (0.009)
15. feature 44 (0.008)
16. feature 24 (0.008)
17. feature 14 (0.008)
18. feature 74 (0.008)
19. feature 33 (0.008)
20. feature 76 (0.007)
21. feature 43 (0.007)
22. feature 89 (0.007)
23. feature 19 (0.007)
24. feature 91 (0.007)
25. feature 77 (0.007)
26. feature 32 (0.007)
27. feature 65 (0.007)
28. feature 39 (0.007)
29. feature 92 (0.007)
30. feature 64 (0.007)
31. feature 45 (0.007)
32. feature 58 (0.007)
33. feature 90 (0.007)
34. feature 35 (0.007)
35. feature 49 (0.007)
36. feature 70 (0.007)
37. feature 71 (0.007)
38. feature 88 (0.007)
39. feature 54 (0.007)
40. feature 18 (0.007)
41. feature 37 (0.007)
42. feature 46 (0.006)
43. feature 67 (0.006)
44. fe

### → feature 110 (disease_type), feature 109 (disease_state) 가 가장 높은 feature importance를 보인다